## 147: Expiración del Token de Verificación 

Agregar un límite de tiempo al token de verificación enviado por correo, para evitar usos tardíos o maliciosos.

🧩 1. Actualizar el modelo User para agregar campo de expiración
python
Copiar
Editar
from datetime import datetime, timedelta

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=False)
    verification_token = Column(String, nullable=True)
    token_expiration = Column(DateTime, nullable=True)  # 🆕 Nuevo campo
🧩 2. Asignar la expiración del token al crearlo
En la función que registra al usuario o genera el token:

python
Copiar
Editar
user.verification_token = secrets.token_urlsafe(32)
user.token_expiration = datetime.utcnow() + timedelta(hours=24)  # ⏰ Token válido por 24 horas
🧩 3. Validar la expiración en la verificación del correo
python
Copiar
Editar
@app.get("/verify/{token}")
def verificar_cuenta(token: str):
    user = db.query(User).filter_by(verification_token=token).first()

    if not user:
        raise HTTPException(status_code=404, detail="Token inválido")

    if user.token_expiration < datetime.utcnow():
        raise HTTPException(status_code=400, detail="Token expirado. Solicita uno nuevo.")

    user.is_active = True
    user.verification_token = None
    user.token_expiration = None
    db.commit()

    return {"mensaje": "✅ Cuenta activada con éxito"}
🧩 4. Al reenviar el correo, también se actualiza el token_expiration
python
Copiar
Editar
user.verification_token = secrets.token_urlsafe(32)
user.token_expiration = datetime.utcnow() + timedelta(hours=24)
db.commit()
enviar_correo_verificacion(user.email, user.verification_token)